In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Multimodal Function Calling with the Gemini API & Python SDK

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/function-calling/multimodal_function_calling.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Ffunction-calling%2Fmultimodal_function_calling.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/function-calling/multimodal_function_calling.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/function-calling/multimodal_function_calling.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>


| | |
|-|-|
|Author(s) | [Kristopher Overholt](https://github.com/koverholt) |

## Overview

### Introduction to Multimodal Function Calling with Gemini

This notebook demonstrates a powerful [Function Calling](https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling) capability of the Gemini model: support for multimodal inputs. With multimodal function calling, you can go beyond traditional text inputs, enabling Gemini to understand your intent and predict function calls based on various inputs like images, audio, video, and PDFs.

Previously, implementing multimodal function calling required two separate calls to the Gemini API: one to extract information from media, and another to generate a function call based on the extracted text. This process was cumbersome, prone to errors, and resulted in the loss of detail in valuable contextual information. Gemini's multimodal function calling capability streamlines this workflow, enabling a single API call that efficiently processes multimodal inputs for accurate function predictions and structured outputs. 

### How It Works

1. **Define Functions and Tools:** Describe your functions, then group them into `Tool` objects for Gemini to use.
2. **Send Inputs and Prompt:** Provide Gemini with multimodal input (image, audio, PDF, etc.) and a prompt describing your request.
3. **Gemini Predicts Action:** Gemini analyzes the multimodal input and prompt to predict the best function to call and its parameters.
4. **Execute and Return:** Use Gemini's prediction to make API calls, then send the results back to Gemini.
5. **Generate Response:** Gemini uses the API results to provide a final, natural language response to the user. 

This notebook will guide you through practical examples of using Gemini's multimodal function calling to build intelligent applications that go beyond the limitations of text-only interactions. 

### Objectives

In this tutorial, you will learn how to use the Vertex AI Gemini API with the Vertex AI SDK for Python to make function calls with multimodal inputs, using the Gemini 1.5 Pro (`gemini-1.5-pro`) model. You'll explore how Gemini can process and understand various input types — including images, video, audio, and PDFs — to predict and execute functions.

You will complete the following tasks:

- Install the Vertex AI SDK for Python.
- Define functions that can be called by Gemini.
- Package functions into tools.
- Send multimodal inputs (images, video, audio, PDFs) and prompts to Gemini.
- Extract predicted function calls and their parameters from Gemini's response.
- Use the predicted output to make API calls to external systems (demonstrated with an image input example). 
- Return API responses to Gemini for natural language response generation (demonstrated with an image input example). 

### Costs

This tutorial uses billable components of Google Cloud:

- Vertex AI

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.


## Getting Started


### Install Vertex AI SDK for Python


In [1]:
# !pip3 install --upgrade --user --quiet google-cloud-aiplatform wikipedia

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [2]:
# # Restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>



### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment. This step is not required if you are using [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).

In [3]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [4]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

## Multimodal Function Calling in Action

### Import libraries


In [5]:
import wikipedia
from IPython.display import display, Markdown
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)

### Image-Based Function Calling: Finding Animal Habitats

In this example, you'll send along an image of a bird and ask Gemini to identify its habitat. This involves defining a function that looks up regions where a given animal is found, creating a tool that uses this function, and then sending a request to Gemini.

First, you define a `FunctionDeclaration` called `get_wildlife_region`. This function takes the name of an animal species as input and returns information about its typical region.

In [6]:
get_wildlife_region = FunctionDeclaration(
    name="get_wildlife_region",
    description="Look up the region where an animal can be found",
    parameters={
        "type": "object",
        "properties": {
            "animal": {"type": "string", "description": "Species of animal"}
        },
    },
)

Next, you create a `Tool` object that includes your `get_wildlife_region` function. Tools help group related functions that Gemini can use:

In [7]:
image_tool = Tool(
    function_declarations=[
        get_wildlife_region,
    ],
)

Now you're ready to send a request to Gemini. Initialize the `GenerativeModel` and specify the image to analyze, along with a prompt. The `tools` argument tells Gemini to consider the functions in your `image_tool`.

In [8]:
model = GenerativeModel("gemini-1.5-pro-001")
generation_config = GenerationConfig(temperature=0)

response = model.generate_content(
   [
       Part.from_uri(
           "gs://github-repo/generative-ai/gemini/function-calling/multi-color-bird.jpg",
            mime_type="image/jpeg",
        ),
        "What is the typical habitat or region where this animal lives?",
    ],
    generation_config=generation_config,
    tools = [image_tool],
)
response_function_call = response.candidates[0].content
response.candidates[0].function_calls[0]

name: "get_wildlife_region"
args {
  fields {
    key: "animal"
    value {
      string_value: "Lilac-breasted Roller"
    }
  }
}

Let's examine the response from Gemini. You can extract the predicted function name:

In [9]:
function_name = response.candidates[0].function_calls[0].name
function_name

'get_wildlife_region'

You can also get the arguments that Gemini predicted for the function call:

In [10]:
function_args = {key: value for key, value in response.candidates[0].function_calls[0].args.items()}
function_args

{'animal': 'Lilac-breasted Roller'}

Now, you'll call an external API (in this case, using the `wikipedia` Python package) using the animal name that Gemini extracted from the image:

In [11]:
api_response = wikipedia.page(function_args["animal"]).content
api_response[:500]

'The lilac-breasted roller (Coracias caudatus) is an African bird of the roller family, Coraciidae. It is widely distributed in Southern and Eastern Africa, and is a vagrant to the southern Arabian Peninsula. It prefers open woodland and savanna, and it is for the most part absent from treeless places. Usually found alone or in pairs, it perches at the tops of trees, poles or other high vantage points from where it can spot insects, amphibians and small birds moving about on the ground. Nesting t'

Finally, you return the API response to Gemini so it can generate a final answer in natural language:

In [12]:
response = model.generate_content(
    [
        Content(
            role="user",
            parts=[
                Part.from_uri(
                    "gs://github-repo/generative-ai/gemini/function-calling/multi-color-bird.jpg",
                    mime_type="image/jpeg",
                ),
                Part.from_text(
                    "Inspect the image and get the regions where this animal can be found",
                )
            ],
        ),
        response_function_call,  # Function call response
        Content(
            parts=[
                Part.from_function_response(
                    name=function_name,
                    response={
                        "content": api_response,  # Return the API response to the Gemini model
                    },
                )
            ],
        ),
    ],
    tools=[image_tool],
)

display(Markdown(response.text))

The lilac-breasted roller (Coracias caudatus) is an African bird. It is widely distributed in Southern and Eastern Africa. 


This example showcases how Gemini's multimodal function calling processes an image, predicts a relevant function and its parameters, and integrates with external APIs to provide comprehensive user information. This process opens up exciting possibilities for building intelligent applications that can "see" and understand the world around them via API calls to Gemini.

### Video-Based Function Calling: Identifying Product Features

Now let's explore how Gemini can extract information from videos for the purpose of invoking a function call. You'll use a video showcasing multiple products and ask Gemini to identify its key features.

Start by defining a function called `get_feature_info` that takes a list of product features as input and could potentially be used to retrieve additional details about those features:

In [13]:
get_feature_info = FunctionDeclaration(
    name="get_feature_info",
    description="Get additional information about a product feature",
    parameters={
        "type": "object",
        "properties": {
            "features": {
                "type": "array",
                "description": "A list of product features",
                "items": {
                    "type": "string",
                    "description": "Product feature"
                },
            }
        },
    },
)

Next, create a tool that includes your `get_feature_info` function:

In [14]:
video_tool = Tool(
    function_declarations=[
        get_feature_info,
    ],
)

Send a video to Gemini, along with a prompt asking for information about the product features, making sure to include your `video_tool` in the `tools` kwarg:

In [15]:
model = GenerativeModel("gemini-1.5-pro-001")
generation_config = GenerationConfig(temperature=0)

response = model.generate_content(
   [
       Part.from_uri(
           "gs://github-repo/generative-ai/gemini/function-calling/made-by-google-24.mp4",
            mime_type="video/mp4",
        ),
        "Inspect the video and get information about the product features shown",
    ],
    generation_config=generation_config,
    tools = [video_tool],
)
response.candidates[0].function_calls

[name: "get_feature_info"
 args {
   fields {
     key: "features"
     value {
       list_value {
         values {
           string_value: "Gemini Live"
         }
         values {
           string_value: "Made You Look"
         }
         values {
           string_value: "Add me"
         }
         values {
           string_value: "Magic Editor"
         }
         values {
           string_value: "Pixel 9 Pro Fold"
         }
         values {
           string_value: "Pixel Buds Pro 2"
         }
         values {
           string_value: "Call Notes"
         }
         values {
           string_value: "Pixel Screenshots"
         }
         values {
           string_value: "Pixel Studio"
         }
       }
     }
   }
 }]

Gemini correctly predicted the `get_feature_info` function:

In [16]:
function_name = response.candidates[0].function_calls[0].name
function_name

'get_feature_info'

And you can see the list of product features that Gemini extracted from the video, which are available as structured function arguments that adhere to the JSON schema we defined in the `FunctionDeclaration`:

In [17]:
function_args = {key: value for key, value in response.candidates[0].function_calls[0].args.items()}
function_args

{'features': ['Gemini Live', 'Made You Look', 'Add me', 'Magic Editor', 'Pixel 9 Pro Fold', 'Pixel Buds Pro 2', 'Call Notes', 'Pixel Screenshots', 'Pixel Studio']}

This example demonstrates Gemini's ability to understand video content. By defining a relevant function, you can use Gemini to extract structured information from videos and perform further actions based on that information.

Now that the multimodal function call response is complete, you could use the function name and function arguments to call an external API using any REST API or client library of your choice, similar to how we did in the previous example with the `wikipedia` Python package.

Since this sample notebook is focused on the mechanics of multimodal function calling rather than the subsequent function calls and API calls, we'll move on to another example with different multimodal inputs. You can refer to other sample notebooks on Gemini Function Calling for more details on where to go from here.

### Audio-Based Function Calling: Generating Book Recommendations

In this example, you'll explore using audio input with Gemini's multimodal function calling. You'll send a podcast episode to Gemini and ask for book recommendations related to the topics discussed.

Define a function called `get_recommended_books` that takes a list of topics as input and (hypothetically) returns relevant book recommendations:

In [18]:
get_recommended_books = FunctionDeclaration(
    name="get_recommended_books",
    description="Get recommended books based on a list of topics",
    parameters={
        "type": "object",
        "properties": {
            "topics": {
                "type": "array",
                "description": "A list of topics",
                "items": {
                    "type": "string",
                    "description": "Topic"
                }
            },
        }
    },
)

Now create a tool that includes your newly defined function:

In [19]:
audio_tool = Tool(
    function_declarations=[
        get_recommended_books,
    ],
)

Provide Gemini with the audio file and a prompt to recommend books based on the podcast content:

In [20]:
model = GenerativeModel("gemini-1.5-pro-001")
generation_config = GenerationConfig(temperature=0)

response = model.generate_content(
   [
       Part.from_uri(
           "gs://github-repo/generative-ai/gemini/function-calling/google-cloud-sre-podcast-s2-e8.mp3",
            mime_type="audio/mpeg",
        ),
        "Inspect the audio file and generate a list of recommended books based on the topics discussed",
    ],
    generation_config=generation_config,
    tools = [audio_tool],
)
response.candidates[0].function_calls

[name: "get_recommended_books"
 args {
   fields {
     key: "topics"
     value {
       list_value {
         values {
           string_value: "Software Reliability"
         }
         values {
           string_value: "Site Reliability Engineering"
         }
         values {
           string_value: "DevOps"
         }
         values {
           string_value: "Cloud Computing"
         }
         values {
           string_value: "System Thinking"
         }
       }
     }
   }
 }]

You can see that Gemini has successfully predicted your `get_recommended_books` function:

In [21]:
function_name = response.candidates[0].function_calls[0].name
function_name

'get_recommended_books'

And the function arguments contain the list of topics that Gemini identified and extracted from the input audio file:

In [22]:
function_args = {key: value for key, value in response.candidates[0].function_calls[0].args.items()}
function_args

{'topics': ['Software Reliability', 'Site Reliability Engineering', 'DevOps', 'Cloud Computing', 'System Thinking']}

This example highlights Gemini's capacity to understand and extract information from audio, enabling you to create applications that respond to spoken content or audio-based interactions.

### PDF-Based Function Calling: Extracting Company Data from Invoices

This example demonstrates how to use Gemini's multimodal function calling to process PDF documents. You'll work with a set of invoices and extract the names of the (fictitious) companies involved.

Define a function called `get_company_information` that (in a real-world scenario) could be used to fetch details about a given list of companies:

In [23]:
get_company_information = FunctionDeclaration(
    name="get_company_information",
    description="Get information about a list of companies",
    parameters={
        "type": "object",
        "properties": {
            "companies": {
                "type": "array",
                "description": "A list of companies",
                "items": {
                    "type": "string", 
                    "description": "Company name"
                },
            }
        },
    },
)

Package your newly defined function into a tool:

In [24]:
invoice_tool = Tool(
    function_declarations=[
        get_company_information,
    ],
)

Now you can provide Gemini with multiple PDF invoices and ask it to get company information:

In [25]:
model = GenerativeModel("gemini-1.5-pro-001")
generation_config = GenerationConfig(temperature=0)

response = model.generate_content(
   [
       Part.from_uri(
           "gs://github-repo/generative-ai/gemini/function-calling/invoice-synthetic-1.pdf",
            mime_type="application/pdf",
        ),
       Part.from_uri(
           "gs://github-repo/generative-ai/gemini/function-calling/invoice-synthetic-2.pdf",
            mime_type="application/pdf",
        ),
       Part.from_uri(
           "gs://github-repo/generative-ai/gemini/function-calling/invoice-synthetic-3.pdf",
            mime_type="application/pdf",
        ),
       Part.from_uri(
           "gs://github-repo/generative-ai/gemini/function-calling/invoice-synthetic-4.pdf",
            mime_type="application/pdf",
        ),
       Part.from_uri(
           "gs://github-repo/generative-ai/gemini/function-calling/invoice-synthetic-5.pdf",
            mime_type="application/pdf",
        ),
        "Inspect the PDF files of invoices and retrieve information about each company",
    ],
    generation_config=generation_config,
    tools = [invoice_tool],
)
response.candidates[0].function_calls

[name: "get_company_information"
 args {
   fields {
     key: "companies"
     value {
       list_value {
         values {
           string_value: "AMNOSH\\nSUPPLIERS"
         }
         values {
           string_value: "BIKBEAR\\nLAW FIRM"
         }
         values {
           string_value: "CUCUONAR\\nAGENCY"
         }
         values {
           string_value: "CWLEY PLC"
         }
         values {
           string_value: "LUVBPTK Company"
         }
       }
     }
   }
 }]

As expected, Gemini predicted the `get_company_information` function:

In [26]:
function_name = response.candidates[0].function_calls[0].name
function_name

'get_company_information'

The function arguments contain the list of company names extracted from the PDF invoices:

In [27]:
function_args = {key: value for key, value in response.candidates[0].function_calls[0].args.items()}
function_args

{'companies': ['AMNOSH\\nSUPPLIERS', 'BIKBEAR\\nLAW FIRM', 'CUCUONAR\\nAGENCY', 'CWLEY PLC', 'LUVBPTK Company']}

This example shows the power of Gemini for processing and extracting structured data from documents, a common requirement in many real-world applications.

### Image-Based Chat: Building a Multimodal Chatbot

Let's put it all together and build a simple multimodal chatbot. This chatbot will understand image inputs and respond to questions using the functions you define.

First, define three functions: `get_animal_details`, `get_location_details`, and `check_color_palette`. These functions represent the capabilities of your chatbot and could potentially be used to retrieve additional details using REST API calls:

In [28]:
get_animal_details = FunctionDeclaration(
    name="get_animal_details",
    description="Look up information about a given animal species",
    parameters={
        "type": "object",
        "properties": {
            "animal": {"type": "string", "description": "Species of animal"}
        },
    },
)

get_location_details = FunctionDeclaration(
    name="get_location_details",
    description="Look up information about a given location",
    parameters={
        "type": "object",
        "properties": {
            "location": {"type": "string", "description": "Location"}
        },
    },
)

check_color_palette = FunctionDeclaration(
    name="check_color_palette",
    description="Check hex color codes for accessibility",
    parameters={
        "type": "object",
        "properties": {
            "colors": {
                "type": "array",
                "description": "A list of colors in hexadecimal format",
                "items": {
                    "type": "string", 
                    "description": "Hexadecimal representation of color, as in #355E3B"
                },
            }
        },
    },
)

Group your functions into a tool:

In [29]:
chat_tool = Tool(
    function_declarations=[
        get_animal_details,
        get_location_details,
        check_color_palette,
    ],
)

Initialize the `GenerativeModel` and start a chat session with Gemini, providing it with your `chat_tool`:

In [30]:
model = GenerativeModel(
    "gemini-1.5-pro-001",
    generation_config=GenerationConfig(temperature=0),
    tools=[chat_tool],
)

chat = model.start_chat()

Send an image of a fox, along with a simple prompt:

In [31]:
response = chat.send_message(
    [
       Part.from_uri(
           "gs://github-repo/generative-ai/gemini/function-calling/baby-fox.jpg",
            mime_type="image/jpeg",
        ),
        "Tell me about this animal",
    ]
)

response.candidates[0].function_calls

[name: "get_animal_details"
 args {
   fields {
     key: "animal"
     value {
       string_value: "fox"
     }
   }
 }]

Now ask about the location details in the image:

In [32]:
response = chat.send_message(
    [
       Part.from_uri(
           "gs://github-repo/generative-ai/gemini/function-calling/baby-fox.jpg",
            mime_type="image/jpeg",
        ),
        "Tell me details about this location",
    ]
)

response.candidates[0].function_calls

[name: "get_location_details"
 args {
   fields {
     key: "location"
     value {
       string_value: "meadow"
     }
   }
 }]

And finally, ask for a color palette based the image:

In [33]:
response = chat.send_message(
    [
       Part.from_uri(
           "gs://github-repo/generative-ai/gemini/function-calling/baby-fox.jpg",
            mime_type="image/jpeg",
        ),
        "Get the color palette of this image and check it for accessibility",
    ]
)

response.candidates[0].function_calls

[name: "check_color_palette"
 args {
   fields {
     key: "colors"
     value {
       list_value {
         values {
           string_value: "#A85C4C"
         }
         values {
           string_value: "#4C8A6C"
         }
       }
     }
   }
 }]

While this chatbot doesn't actually execute the predicted functions, it demonstrates creating an interactive experience using multimodal inputs and function calling in a chat format. You can extend this example by implementing REST API calls or client library requests for each function to create a truly functional and engaging multimodal chatbot that's connected to the real world.

## Conclusions

In this notebook, you explored the powerful capabilities of Gemini's multimodal function calling. You learned how to:

- Define functions and package them into tools.
- Send multimodal inputs (images, video, audio, PDFs) and prompts to Gemini. 
- Extract predicted function calls and their parameters.
- Use the predicted output to make (or potentially make) API calls.
- Return API responses to Gemini for natural language generation. 

You've seen how Gemini can understand and act on a range of different multimodal inputs, which opens up a world of possibilities for building innovative and engaging multimodal applications.  You can now use these powerful tools to create your own intelligent applications that seamlessly integrate media, natural language, and calls to external APIs and system.

Experiment with different modalities, functions, and prompts to discover the full potential of Gemini's multimodal and function calling capabilities. And you can continue learning by exploring other sample notebooks in this repository and exploring the [documentation for Gemini Function Calling](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling). 